In [50]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate

from langchain.output_parsers import StructuredOutputParser, ResponseSchema


from langchain_community.document_loaders import PyMuPDFLoader


## Definition der LLMS

In [2]:
gpt_model = 'gpt-4-1106-preview'
embedding_model = 'text-embedding-3-small'

In [32]:
llm = ChatOpenAI(temperature=0, model_name=gpt_model)
embeddings = OpenAIEmbeddings()#(model=embedding_model) 

## Loader Experimente

In [46]:
pdf_file = "grundlegendes/AS_611008_template.pdf"
pdf_file2 = "grundlegendes/AS_461006_template 1.pdf"

In [47]:
pdfloader = PyMuPDFLoader(pdf_file2)
data = pdfloader.load()

antrag= data[0].page_content

antrag

' \nAnmeldung zum Kindergarten, zur Schulkindbetreuung, Tagesstätte /\nÜbermittagbetreuung, Blocköffnungszeit, kleinen altersgemischten\nGruppe\n \nGrundlegende Fragen zum Anmeldung\nBitte machen Sie mindestens eine Angabe.\nzum Kindergarten\nzur Schulkinderbetreuung\nzur Tagesstätte / Übermittagbetreuung\nzur Blocköffnungszeit\nzur kleinen altersgemischten Gruppe\nab Datum\nHaben Sie Interesse, im Elternbeirat mitzuarbeiten?\nja\nnein\nDürfen wir Ihre Telefonnummer in der Kindergartenliste veröffentlichen?\nja\nnein\nDürfen wir das Kind fotografieren / filmen?\nja\nnein\nKönnen Sie angaben zum Hausarzt des Kindes machen?\nja\nnein\nAntragstellende Person\nAnrede\nFrau\nHerr\nPerson\nVornamen\nFamilienname\nGegebenenfalls Geburtsname\nWohnanschrift\nStaat\nPostleitzahl\nOrt\nStraße\nHausnummer\nKontaktdaten\nTelefon (Mobil)\nTelefon (Festnetz oder Mobil)\nE-Mail\nSeite 1 von 7\nPDF erstellt mit: www.form-solutions.de\nArtikel-Nr.: AS_461006\n'

## PromptTemplate und ResponseSchema

In [49]:
template = """Sie sind ein hilfsbereiter, respektvoller und ehrlicher Assistent, der Online-Formulare den Benutzern erklärt. Antworten Sie immer so hilfreich wie möglich und gleichzeitig sicher auf Deutsch. Wenn Sie keine Antwort wissen, versuchen Sie nicht eine Antwort zu erfinden.
    Befolge die folgenden 3 Schritte:
    1. Lies das Formular unten.
    2. Beantworte die Frage mit Hilfe der Informationen aus dem Formular auf Deutsch, benutze kein Englisch.
    3. Verwende nur Informationen aus dem Formular.
    
    Gebe mir bitte Ausfüllhinweise für das Formular.

    Formular : {context}
      
    
    Ausgabeformat: {format_instructions}"""
   

response_schemas = [
    ResponseSchema(name="grundlegende_Angaben", description="schreibe hier die grundlegenden Angaben, die in dem Formular gemacht werden müssen, rein."),
    ResponseSchema(
        name="erforderliche_Unterlagen",
        description="Hier werden die Unterlagen, Anhänge und Nachweise vermerkt, die mit dem Formular abgegeben werden müssen oder können.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
        input_variables=["context"], 
        template = template,
        partial_variables={"format_instructions": format_instructions},
    )    

chain = (
    prompt |
    llm |
    output_parser
)

response = chain.invoke({"context": antrag})

response

{'grundlegende_Angaben': 'Bitte wählen Sie mindestens eine der folgenden Optionen aus: zum Kindergarten, zur Schulkinderbetreuung, zur Tagesstätte / Übermittagbetreuung, zur Blocköffnungszeit, zur kleinen altersgemischten Gruppe. Geben Sie das Datum an, ab wann die Anmeldung gelten soll. Entscheiden Sie, ob Sie im Elternbeirat mitarbeiten möchten, ob Ihre Telefonnummer veröffentlicht werden darf, ob das Kind fotografiert / gefilmt werden darf und ob Sie Angaben zum Hausarzt des Kindes machen können. Füllen Sie die persönlichen Daten der antragstellenden Person aus, inklusive Anrede, Vornamen, Familienname, gegebenenfalls Geburtsname, Wohnanschrift (Staat, Postleitzahl, Ort, Straße, Hausnummer) und Kontaktdaten (Telefon Mobil, Telefon Festnetz oder Mobil, E-Mail).',
 'erforderliche_Unterlagen': 'Das Formular gibt keine spezifischen Informationen über erforderliche Unterlagen, Anhänge oder Nachweise, die mit dem Formular abgegeben werden müssen oder können.'}